# Reduce WAXS

In [ ]:
from pathlib import Path
import pyFAI
import fabio
import numpy as np
import yaml
import matplotlib.pylab as plt
from matplotlib.colors import LogNorm
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

FIG_SIZE = (7.5, 3.75)

raw_stitch = fabio.open("raw-stitched-data.tif").data.astype(np.float64)
time_pixel = fabio.open("stitched-exposure-time.tif").data.astype(np.float64)
with open("params.yaml", "r") as yf:
    params = yaml.safe_load(yf)
    exposure_time = params["exposure"]
print(f"Exposure time  = {exposure_time} s / (2 images)")

fig = plt.figure(figsize=FIG_SIZE, facecolor="w")
ax1 = plt.subplot()
pos = ax1.imshow(raw_stitch, norm=LogNorm(1, np.max(raw_stitch)))
ax1.set_title("Transformed image")
ax1.set_xlabel("column (pixels)")
ax1.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax1, shrink=0.7)

fig = plt.figure(figsize=FIG_SIZE, facecolor="w")
ax1 = plt.subplot()
pos = ax1.imshow(time_pixel)
ax1.set_title("Exposure time per pixel (s)")
ax1.set_xlabel("column (pixels)")
ax1.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax1, shrink=0.7)

## Adust pixel weights

In [ ]:
data = raw_stitch * exposure_time / time_pixel
data[np.where(data == np.infty)] = 0
data = np.nan_to_num(data)
print(data.max())

fig = plt.figure(figsize=FIG_SIZE, facecolor="w")
ax1 = plt.subplot()
pos = ax1.imshow(data, norm=LogNorm(1, np.max(data)))
ax1.set_title("Transformed image with exposure times adusted")
ax1.set_xlabel("column (pixels)")
ax1.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax1, shrink=0.7)
plt.savefig("stitch.png", dpi=1000)

## Create Cake and reduce data
You must have a `cal-transformed.poni` file. This can be made using pyFAI-calib2 GUI

You can create a mask using pyFAI-calib2 GUI

In [ ]:
ai = pyFAI.load("cal.poni")

try:
    mask = fabio.open("mask.edf").data.astype(bool)
    print("Used mask")
except FileNotFoundError:
    mask = None
    print("Did not load mask, you can create one using pyFAI-calib2")

q_bins = 1000
azimuthal_bins = 1800
cake = ai.integrate2d_ng(data, q_bins, azimuthal_bins, mask=mask, unit="q_A^-1")

q_bins = 5000
redu = ai.integrate1d_ng(data, q_bins, mask=mask, unit="q_A^-1", filename="reduction.edf")

## Plot reductions

In [ ]:
fig = plt.figure(figsize=(5, 3), facecolor="w")
ax1 = plt.subplot()
pos = ax1.imshow(cake[0], norm=LogNorm(1, np.max(cake[0])),
                 extent=(np.min(cake[1]), np.max(cake[1]), 360, 0),
                 aspect='auto')
ax1.set_title("Cake")
ax1.set_xlabel(r"$q\ (\mathregular{\AA}^{-1})$")
ax1.set_ylabel(r"$\psi$ (degrees)")
ax1.set_yticks(np.arange(0, 361, 40))
ax1.yaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
# ax1.set_xticks(np.arange(0, np.min(cake[1]) + np.max(cake[1]), 0.5))
ax1.xaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
fig.colorbar(pos, ax=ax1)
plt.savefig("cake.png", dpi=1000, bbox_inches="tight")

plt.figure(figsize=(4, 3))
ax2 = plt.subplot()
ax2.scatter(redu[0], redu[1], s=1, marker="o", color="black", alpha=0.5)
ax2.set_title("Reduction")
ax2.xaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
# ax2.set_xticks(np.arange(0, np.min(redu[0]) + np.max(redu[0]), 0.5))
ax2.yaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
ax2.set_xlabel(r"$q\ (\mathregular{\AA}^{-1})$")
ax2.grid()
ax2.set_ylabel("Intensity")
plt.savefig("reduction.png", dpi=1000, bbox_inches="tight")